In [ ]:
from pulp import *
#from geopy.distance import geodesic
from pydantic import BaseModel
#import geopandas as gpd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
from typing import List, Optional, Union

### Define our Classes

In [ ]:
class Technology(BaseModel):
    pass

class GenerationTechnology(Technology):
    name: str
    construction_lead_years : int = 1 # years
    emissions_rate: float             # t / MW-year
    area_footprint: float             # sqkm / MW
    capital_cost_var: float           # $/kW
    capital_cost_fixed: float         # $
    capital_cost_gr: float            # e.g. -0.05 -> 5% decrease/yr
    opex: float                       # $ / kW
    lifespan: int                     # number of years
    
    
class TransmissionTechnology(Technology):
    name: str
    construction_lead_years : int = 1  # years
    loss: float                        # % / km
    capital_cost_var: float            # $/KW/km
    capital_cost_fixed: float          # $
    capital_cost_gr: float = 0.        # e.g. 0.06 -> 6% increase/yr
    lifespan: int                      # number of years
    
class InitialTech(Technology):
    ob_type: str
    ob_key: Union[str,tuple]
    tech_type: str
    capacity: float # kw
    age: int        # years
    
class Land(BaseModel):
    name: str
    cost: float                        # $ / sqm

class Node(BaseModel):
    id: str
    solar_irradiance: float            # kWh/kW
    area: float                        # sqkm
    initial_landarea: dict              # dict(lc_key:percentage)
    initial_population: int             # people
    node_technologies: List[GenerationTechnology]
    
class Edge(BaseModel):
    id: tuple
    distance: float                    # km
    edge_technologies: List[TransmissionTechnology]
    
class GrowthRate(BaseModel):
    max_rate: float
    min_rate:float 
    peak_year:int
    half_decay:int
    
class GridlandiaGlobals(BaseModel):
    population_gr: GrowthRate
    final_energy_intensity: int # kWh/person/yr
    final_energy_intensity_gr: GrowthRate 
    
class GridLandiaConstraints(BaseModel):
    net_zero_year: Optional[int]
    land_constraints: Optional[int]

In [ ]:
class GridWorld(object):
    
    def __init__(self, node_gdf:gpd.GeoDataFrame, column_labels: dict):
        pass

In [ ]:
class Gridlandia(object):
    def __init__(
        self, 
        nodes: List[Nodes], 
        edges: List[Edges],
        generation_technologies: List[GenerationTechnologies], 
        transmission_technologies: List[TransmissionTechnologies],, 
        global_params: GridlandiaGlobals,
    ):
        

### Setup our problem

In [ ]:
24000 # MJ/t
300 # $/t
0.2778 # kWh/MJ
300/24000/.2778*3*8760 # $/kWh_th

In [ ]:
# demand 1100 kWh//yr/person
1100/8760

In [ ]:
# $50/2000lbs -> $55/t -> 6150 kWh_t/t -> 8.943 $/MWh_t
300/24000/.2778*3*8760

In [ ]:
# 8.943*3 -> 26.829 $/MWh_e

In [ ]:
26.829*8760 # $ / MW / yr

In [ ]:
300/24000/.2778*3*8760

In [ ]:
# generation technologies
generation_techs = dict(
    utility_solar = dict(
        emissions_rate=0,
        area_footprint = 11300/1000/1000*3, # sqkm/MW (covered_area * tot_area_fac)
        capital_cost_var = 4000000, # $/MW + land costs
        capital_cost_fixed=5e5, # $/yr
        capital_cost_gr = -0.1,
        opex=0,
        lifespan=40,
    ),
    distributed_solar = dict(
        emissions_rate =0,
        area_footprint = 0,
        capital_cost_var = 6000000, # $/MW
        capital_cost_fixed = 1e3,
        capital_cost_gr = -0.1,
        opex=0,
        lifespan=30,
    ),
    coal_thermal = dict(
        construction_lead_years=5,
        emissions_rate=8760, # 1t/MWh*8760hrs
        area_footprint=0,
        capital_cost_var=600000, # $/MW
        capital_cost_fixed = 1e6,
        capital_cost_gr = 0.,
        opex = 26.829,#*8760, # $/MW_e/yr
        lifespan=35,
    )
)       

In [ ]:
# transmission technologies
transmission_techs = dict(
    distribution = dict(
        loss=0.001,       # 0.1% / km
        capital_cost_var=10/1000, # $/Mw/km
        capital_cost_fixed=1e3,
        lifespan=30,
    ),
    transmission_220 = dict(
        loss=0.0004,
        capital_cost_var = 20/1000, # $/MW/km
        capital_cost_fixed=1e5,
        lifespan=40,
    ),
    transmission_660 = dict(
        construction_lead_years=2,
        loss=0.0001,
        capital_cost_var = 40/1000, # $/MW/km
        capital_cost_fixed=1e6,
        lifespan=40
    )
)

In [ ]:
# landcover/use types:
land_uses = dict(
    builtup = dict(cost=70),
    forest = dict(cost=30),
    barren = dict(cost=10),
    agriculture = dict(cost=15),
    plantation = dict(cost=35),
)

In [ ]:
gridland_globals = GridlandiaGlobals(
    population_gr= GrowthRate(max_rate=0.015, min_rate=0.003, peak_year=2025,half_decay=2040),
    final_energy_intensity=1,
    final_energy_intensity_gr=GrowthRate(max_rate=0.04, min_rate=0.005, peak_year=2030,half_decay=2050),
)

In [ ]:
gridland_globals.population_gr.__dict__

In [ ]:
# peak growth year; peak gr, baseline gr, 

In [ ]:
def _sigmoid(x):
    return np.exp(x) / (1+np.exp(x))

In [ ]:
def D_sigmoid(x):
    return _sigmoid(x)*(1-_sigmoid(x))

In [ ]:
def growth_rate_bell(max_rate, min_rate, peak_year, half_decay):
    """ return a function which can calculate a rate in a given year with a curve with the input params"""
    
    def gr_fn(x):
        return min_rate + D_sigmoid((x-peak_year)*1.76274/(half_decay-peak_year))/0.25*(max_rate-min_rate)
    
    return gr_fn

### Test our solvers

In [ ]:
pulpTestAll()

### Setup our problem
- build nodes
- build edges
- Mixin special settings
- set initial tech
- declare vars
- declare constraints
- declare affines

In [ ]:
problem = Gridlandia(nodes, edges, etc)
problem.solve()
for var in problem.vars:

**todo:**
- max capacity add
- land use constraint
- land use simulation

## Toyest toy: Two nodes capacity planning

In [ ]:
all_gen_techs = [GenerationTechnology(name=kk,**vv) for kk,vv in generation_techs.items()]

In [ ]:
all_transmission_tech = [TransmissionTechnology(name=kk,**vv) for kk,vv in transmission_techs.items()]

In [ ]:
initial_techs = [
    InitialTech(
        ob_type='node',
        ob_key='node_1',
        tech_type='coal_thermal',
        capacity=15,
        age=15
    ),
    InitialTech(
        ob_type='edge',
        ob_key=('node_1','node_2'),
        tech_type='distribution',
        capacity=15,
        age=20
    ),
]

In [ ]:
node_1 = Node(
    id='node_1',
    solar_irradiance=800, 
    area=50,
    initial_landarea=dict(meow=1.),
    initial_population=1e5,
    node_technologies= all_gen_techs,
)

In [ ]:
node_2 = Node(
    id='node_2',
    solar_irradiance=1400, 
    area=100,
    initial_landarea=dict(meow=1.),
    initial_population=5e4,
    node_technologies= all_gen_techs,
)

In [ ]:
edge_1_2 = Edge(
    id=('node_1','node_2'),
    distance=100,
    edge_technologies=all_transmission_tech
)

In [ ]:
nodes = {node.id:node for node in [node_1, node_2]}
edges = {edge.id:edge for edge in [edge_1_2]}

In [ ]:
years = list(range(2020,2070))

In [ ]:
### declare model
model = LpProblem("energy_capacity_least_costs",LpMinimize)

In [ ]:
from collections import defaultdict
from itertools import product

In [ ]:
def fill_product(d, list_a, list_b, list_c, default_val):
    for el_a in list_a:
        d[el_a] = {}
        for el_b in list_b:
            if list_c is None:
                d[el_a][el_b] = default_val
            else:
                d[el_a][el_b] = {}
                for el_c in list_c:
                    d[el_a][el_b][el_c] = default_val
    return d

In [ ]:
# setup vars: ini capacities
initial_node_capacity = fill_product({}, nodes.keys(), generation_techs.keys(),years, 0)
initial_edge_capacity = fill_product({}, edges.keys(), transmission_techs.keys(),years, 0)

    
for initial_tech in initial_techs:
    if initial_tech.ob_type=='node':
        for year in range(min(years),min(years)+generation_techs[initial_tech.tech_type]['lifespan']-initial_tech.age):
            initial_node_capacity[initial_tech.ob_key][initial_tech.tech_type][year] += initial_tech.capacity
    elif initial_tech.ob_type=='edge':
        for year in range(min(years),min(years)+transmission_techs[initial_tech.tech_type]['lifespan']-initial_tech.age):
            initial_edge_capacity[initial_tech.ob_key][initial_tech.tech_type][year] += initial_tech.capacity

In [ ]:
# setup vars: node demands
population_gr_years = growth_rate_bell(**gridland_globals.population_gr.__dict__)(np.array(years))
energy_intensity_gr_years = growth_rate_bell(**gridland_globals.final_energy_intensity_gr.__dict__)(np.array(years))

node_demand = fill_product({}, nodes.keys(), years,None,0)
for node_id in nodes.keys():
    for ii_y, year in enumerate(years):
        # node_demand = pop_in_node_yr * energy_intensity_in_node_yr / 8760
        pop_in_node_yr = nodes[node_id].initial_population*np.prod((1.+population_gr_years[:ii_y]))
        energy_intensity_in_node_yr = gridland_globals.final_energy_intensity*np.prod((1.+energy_intensity_gr_years[:ii_y]))
        node_demand[node_id][year] = pop_in_node_yr * energy_intensity_in_node_yr / 8760

In [ ]:
all_distances = {kk:vv.distance for kk,vv in edges.items()}
all_distances.update({(kk[1],kk[0]):vv.distance for kk,vv in edges.items()})

In [ ]:
### variables
# flows in [edges, years]
#F = LpVariable.dicts("Flows",(edges.keys(),transmission_techs.keys(),years), cat='continuous')
reverse_edge_keys = [(e[1],e[0]) for e in edges.keys()]
F = LpVariable.dicts("Flows",(reverse_edge_keys+list(edges.keys()),transmission_techs.keys(),years), lowBound=0, cat='continuous')
# capacities_additions in [nodes, technologies, year]
Cap_add_nodes = LpVariable.dicts("Cap_Add_nodes", (nodes.keys(), generation_techs.keys(), years), lowBound=0, cat='continuous')
# capacity_additions in [edges, technologies, year]
Cap_add_edges = LpVariable.dicts("Cap_Add_edges", (edges.keys(), transmission_techs.keys(), years), lowBound=0, cat='continuous')
# construction transit vars
Construction_node_bool = LpVariable.dicts("Constr_nodes", (nodes.keys(), generation_techs.keys(), years), 0,1, LpInteger)
Construction_edge_bool = LpVariable.dicts("Constr_edges", (edges.keys(), transmission_techs.keys(), years), 0,1, LpInteger)
Flow_direction_bool = LpVariable.dicts("Flow_direction", (edges.keys(), transmission_techs.keys(), years), 0,1, LpInteger)
# Generation in [nodes, technologies, year]
Gen_nodes = LpVariable.dicts("Gen", (nodes.keys(), generation_techs.keys(), years), lowBound=0, cat='continuous')

# construction boolean forcing

In [ ]:
# build affines
supply_capacities = {}
for node_id, tech_id, year in product(nodes.keys(),generation_techs.keys(),years):
    # initial + additional - retirement
    supply_capacities[(node_id,tech_id,year)] = initial_node_capacity[node_id][tech_id][year] + sum([Cap_add_nodes[node_id][tech_id][build_year] for build_year in range(max(min(years),year-generation_techs[tech_id]['lifespan']),year+1)])

transmission_capacities = {}
for edge_id, tech_id, year in product(edges.keys(),transmission_techs.keys(),years):
    # initial + additional - retirement
    transmission_capacities[(edge_id,tech_id,year)] = initial_edge_capacity[edge_id][tech_id][year] + sum([Cap_add_edges[edge_id][tech_id][build_year] for build_year in range(max(min(years),year-transmission_techs[tech_id]['lifespan']),year+1)])
    
supply = {}
for node_id, year in product(nodes.keys(), years):
    supply[(node_id, year)] = sum([Gen_nodes[node_id][tech_id][year] for tech_id in generation_techs.keys()])
    
netflow = {}
for node_id, year in product(nodes.keys(), years):
    # sum( inflow*eff - outflow)
    #
    netflow[(node_id,year)] = sum([F[e][t][year]*(1-transmission_techs[t]['loss']*all_distances[e])  for e in reverse_edge_keys+list(edges.keys()) for t in transmission_techs.keys() if e[1]==node_id]) - \
                             sum([F[e][t][year] for e in reverse_edge_keys+list(edges.keys()) for t in transmission_techs.keys() if e[0]==node_id])

In [ ]:
### contraints
# demand satisficing and conservation -> supply in techs + netflow - demand =0 in (nodes, years)
for node_id, year in product(nodes.keys(), years):
    model += supply[(node_id, year)] + netflow[(node_id,year)] - node_demand[node_id][year] == 0

In [ ]:
# supply capacity
for n, t, year in product(nodes.keys(), generation_techs.keys(), years):
    model += Gen_nodes[n][t][year]<=supply_capacities[(n,t,year)]

In [ ]:
# edge flow capacity
for e, t, year in product(edges.keys(), transmission_techs.keys(), years):
    model += F[e][t][year]<=transmission_capacities[(e,t,year)]
    # constrain the reverse direction also
    model += F[(e[1],e[0])][t][year]<=transmission_capacities[(e,t,year)]

In [ ]:
# total supply >= total demand
for year in years:
    model += sum([supply[(node_id,year)] for node_id in nodes.keys()]) >= sum([node_demand[node_id][year] for node_id in nodes.keys()])

In [ ]:
M = max([max(d.values()) for d in list(node_demand.values())])*1000 # a big number

In [ ]:
# edge flow direction
for e, t, year in product(edges.keys(), transmission_techs.keys(), years):
    model += F[e][t][year] <= M*Flow_direction_bool[e][t][year]
    model += F[(e[1],e[0])][t][year] <= M*(1-Flow_direction_bool[e][t][year])

In [ ]:
# construction forcing boolean
# https://cs.stackexchange.com/questions/69531/greater-than-condition-in-integer-linear-program-with-a-binary-variable

# C=0

for n, t, year in product(nodes.keys(), generation_techs.keys(), years):
    model += Cap_add_nodes[n][t][year] >= 1+-M*(1-Construction_node_bool[n][t][year]) # B≥C+1−M(1−A)
    model += Cap_add_nodes[n][t][year] <= M*Construction_node_bool[n][t][year] # B≤C+MA
    
for e, t, year in product(edges.keys(), transmission_techs.keys(), years):
    model += Cap_add_edges[e][t][year] >= 1+-M*(1-Construction_edge_bool[e][t][year]) # B≥C+1−M(1−A)
    model += Cap_add_edges[e][t][year] <= M*Construction_edge_bool[e][t][year] # B≤C+MA

In [ ]:
# cost-years
# simply for now, just fixed + variable in cap add
cost = {}
for year in years:
    cost[year] = sum([Cap_add_nodes[n][gt][year]*generation_techs[gt]['capital_cost_var'] for n,gt in product(nodes.keys(), generation_techs.keys())]) + \
                 sum([Cap_add_edges[e][tt][year]*transmission_techs[tt]['capital_cost_var'] for e,tt in product(edges.keys(), transmission_techs.keys())]) + \
                 sum([Construction_node_bool[n][gt][year]*generation_techs[gt]['capital_cost_fixed'] for n,gt in product(nodes.keys(), generation_techs.keys())]) + \
                 sum([Construction_edge_bool[e][tt][year]*transmission_techs[tt]['capital_cost_fixed'] for e,tt in product(edges.keys(), transmission_techs.keys())]) + \
                 sum([Gen_nodes[n][gt][year]*generation_techs[gt]['opex'] for n,gt in product(nodes.keys(),generation_techs.keys())])

In [ ]:
total_cost = sum([cost[year] for year in years])

In [ ]:
model += total_cost, "Total Cost"

In [ ]:
model.solve(GUROBI_CMD())

In [ ]:
print(LpStatus[model.status])

In [ ]:
for kk,vv in cost.items():
    print (kk,vv.value())

In [ ]:
node_demand

In [ ]:
for kk,vv in netflow.items():
    print (kk, vv.value())

In [ ]:
for kk,vv in Cap_add_edges.items():
    for kk2, vv2 in vv.items():
        for kk3, vv3 in vv2.items():
            print (kk,kk2,kk3,vv3.value())

In [ ]:
for kk,vv in F.items():
    for kk2, vv2 in vv.items():
        for kk3, vv3 in vv2.items():
            print (kk,kk2,kk3,vv3.value())

In [ ]:
for kk,vv in Gen_nodes.items():
    for kk2, vv2 in vv.items():
        for kk3, vv3 in vv2.items():
            print (kk,kk2,kk3,vv3.value())
    #print (kk,vv.keys())

In [ ]:
for kk,vv in Cap_add_nodes.items():
    for kk2, vv2 in vv.items():
        for kk3, vv3 in vv2.items():
            print (kk,kk2,kk3,vv3.value())
    #print (kk,vv.keys())

In [ ]:
for kk,vv in Cap_add_edges.items():
    for kk2, vv2 in vv.items():
        for kk3, vv3 in vv2.items():
            print (kk,kk2,kk3,vv3.value())
    #print (kk,vv.keys())

In [ ]:
for kk,vv in Construction_node_bool.items():
    for kk2, vv2 in vv.items():
        for kk3, vv3 in vv2.items():
            print (kk,kk2,kk3,vv3.value())
    #print (kk,vv.keys())